<a href="https://colab.research.google.com/github/luiz826/Sistema_de_recomendacao/blob/main/Recomendation_Sys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de Recomendação para filmes da netflix

### Importação das bibliotecas

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/luiz826/Site_Sistema_de_recomendacao/main/netflix_titles.csv")

In [3]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


### Pré-processamento dos dados

##### Selecionando apenas os filmes e as colunas mais importantes

In [4]:
mov = df.query("type == 'Movie'")

In [5]:
movies = mov[["title", "director", "cast", "country", "rating", "listed_in"]].reset_index()

##### Essa função serve para binarizar as palavras 

In [7]:
def binariza(col, classe):
    classe[col] = classe[col].fillna("Not Informed")
    
    lista = []
    for i in classe[col]:
        lista_aux = i.split(",")
        for j in lista_aux:
            lista.append(j)

    lista_sort = sorted(set(lista))

    binary_lista = [[0] * 0 for i in range(len(set(lista)))]

    for i in classe[col]:
        c = 0
        for j in lista_sort:
            if j in i:
                binary_lista[c].append(1.0)
            else:
                binary_lista[c].append(0.0)
            c+=1

    return pd.DataFrame(binary_lista).T

In [8]:
bin_actors = binariza("cast", movies)

In [9]:
bin_director = binariza("director", movies)

In [10]:
bin_country = binariza("country", movies)

In [11]:
bin_rating = binariza("rating", movies)

In [12]:
bin_listed_in = binariza("listed_in", movies)

##### Neste momento eu faço a concatenação das colunas e utilizo a similaridade de cossenos para encontrar a similaridade entre os binários

In [13]:
bin_df = pd.concat([bin_actors, bin_director, bin_country, bin_rating, bin_listed_in], axis=1, ignore_index=True)

In [14]:
cs_df = pd.DataFrame(cosine_similarity(bin_df))

##### Definindo a função para gerar recomendações

In [27]:
def recomendation(movie, n=5):
    if movie in movies["title"].values:
        idm = movies.loc[movies["title"] == movie].index.item()
        rm = cs_df[idm].sort_values(ascending=False).head(n+1)[1:]
        movies_c = movies.copy()
        movies_c["CS"] = cs_df[idm].sort_values(ascending=False)
        return movies_c.iloc[list(rm.index)]
    
    print("This movie is not in our database...\n")
    print("or did you mean: ", end="")
    for i in movies["title"].values:
        if movie in i:
            print(i)
    

In [28]:
recomendation("Tom and Jerry: The Magic Ring")

,index,title,director,cast,country,rating,listed_in,CS
2573,3672,Little Singham: Kaal Ki Tabaahi,Prakash Satam,Not Informed,Not Informed,TV-Y7,"Children & Family Movies, Comedies",0.632456
2571,3670,Little Singham in London,Prakash Satam,Not Informed,Not Informed,TV-Y7,"Children & Family Movies, Comedies",0.632456
2574,3673,Little Singham: Legend of Dugabakka,Not Informed,Not Informed,Not Informed,TV-Y7,"Children & Family Movies, Comedies",0.632456
2569,3668,Little Singham aur Kaal ka Mahajaal,Prakash Satam,Not Informed,Not Informed,TV-Y7-FV,"Children & Family Movies, Comedies",0.603023
1508,2157,Figaro Pho,Luke Jurevicius,Luke Jurevicius,United Kingdom,TV-Y7,"Children & Family Movies, Comedies",0.553399


In [29]:
recomendation("Pulp Fiction")

,index,title,director,cast,country,rating,listed_in,CS
3326,4776,Paris Is Burning,Jennie Livingston,Not Informed,United States,R,"Classic Movies, Cult Movies, Documentaries",0.422116
131,165,A Clockwork Orange,Stanley Kubrick,"Malcolm McDowell, Patrick Magee, Michael Bates...","United Kingdom, United States",R,"Classic Movies, Cult Movies, Dramas",0.402492
940,1288,Chasing Amy,Kevin Smith,"Ben Affleck, Joey Lauren Adams, Jason Lee, Dwi...",United States,R,"Comedies, Cult Movies, Dramas",0.377124
2795,4008,Mean Streets,Martin Scorsese,"Robert De Niro, Harvey Keitel, David Proval, A...",United States,R,"Classic Movies, Dramas, Independent Movies",0.367065
5258,7609,Who's That Knocking at My Door?,Martin Scorsese,"Zina Bethune, Harvey Keitel, Anne Collette, Le...",United States,R,"Classic Movies, Dramas, Independent Movies",0.367065


In [30]:
recomendation("To All the Boys I’ve Loved Before")

,index,title,director,cast,country,rating,listed_in,CS
4929,7112,To All the Boys: P.S. I Still Love You,Michael Fimognari,"Lana Condor, Noah Centineo, Jordan Fisher, Ann...",United States,TV-14,"Comedies, Romantic Movies",0.744387
3752,5386,Santo Cachón,Not Informed,Not Informed,Colombia,TV-14,"Comedies, International Movies, Romantic Movies",0.514259
3197,4597,Okafor's Law,Omoni Oboli,Not Informed,Nigeria,TV-14,"Comedies, International Movies, Romantic Movies",0.514259
1190,1692,Deliha,Hakan Algül,"Gupse Özay, Barış Arduç, Cihan Ercan",Turkey,TV-14,"Comedies, International Movies, Romantic Movies",0.473050
5241,7586,When We First Met,Ari Sandel,"Adam DeVine, Alexandra Daddario, Shelley Henni...",United States,TV-14,"Comedies, Romantic Movies",0.465379


In [31]:
recomendation("The Two Popes")

,index,title,director,cast,country,rating,listed_in,CS
1069,1501,Cops and Robbers,"Arnon Manor, Timothy Ware-Hill",Timothy Ware-Hill,United States,PG-13,Dramas,0.481125
1937,2794,Honey: Rise Up and Dance,Bille Woodruff,"Teyana Taylor, Sierra Aylina McClain, Bryshere...",United States,PG-13,Dramas,0.456435
1260,1793,Doctor Zhivago,David Lean,"Omar Sharif, Julie Christie, Geraldine Chaplin...","United States, Italy, United Kingdom, Liechten...",PG-13,"Classic Movies, Dramas, Romantic Movies",0.452911
3628,5227,Rize,David LaChapelle,"Tommy the Clown, Tight Eyez","United States, United Kingdom",PG-13,Documentaries,0.435194
4563,6594,The Life Ahead,Edoardo Ponti,"Sophia Loren, Ibrahima Gueye, Renato Carpentie...",Italy,PG-13,"Dramas, International Movies",0.420084
